Manipulators are verbs

In [1]:
class Color(str: String = "")

class Pixel(x: Int, y: Int) {
  def paint(color: Color): Unit = {
    // Implementation here
  }
}

val center = new Pixel(50, 50)
center.paint(new Color("red"))

defined class Color
defined class Pixel
center: Pixel = ammonite.$sess.cmd1$Helper$Pixel@34998709

When an object allows us to manipulate, the name has to be a verb, and there must be no return value.

Examples of refactoring


In [1]:
class Document {
  def write(content: java.io.InputStream): Unit = {
    // Implementation here
  }
}

defined class Document

The example above shows a Document class with a write method that serves as a manipulator. Following the principle, it has a verb name and returns Unit (void).

However, what if we need to know how many bytes were saved? Let's refactor according to the principles:


In [2]:
// Problem: This violates our principle - a manipulator returning a value
class DocumentProblem {
  def write(content: java.io.InputStream): Int = {
    // Implementation that writes and returns bytes written
    42 // Dummy return for example
  }
}

// Solution: Separate the concerns
class Document {
  def output(): OutputPipe = {
    new OutputPipe()
  }
}

class OutputPipe {
  def write(content: java.io.InputStream): Unit = {
    // Implementation here
  }

  def bytes: Int = {
    // Return number of bytes written
    42 // Dummy return for example
  }

  def time: Long = {
    // Return time taken for writing
    123L // Dummy return for example
  }
}

// Usage example
val doc = new Document()
val pipe = doc.output()
pipe.write(new java.io.ByteArrayInputStream("Hello".getBytes))
val bytesWritten = pipe.bytes
val timeElapsed = pipe.time

defined class DocumentProblem
defined class Document
defined class OutputPipe
doc: Document = ammonite.$sess.cmd2$Helper$Document@69ee3a5e
pipe: OutputPipe = ammonite.$sess.cmd2$Helper$OutputPipe@6d2ee5ef
bytesWritten: Int = 42
timeElapsed: Long = 123L

Notice how we've refactored to maintain the principle:
1. `output()` is a builder - it returns a new object and has a noun name
2. `write()` is a manipulator - it has a verb name and returns Unit
3. `bytes` and `time` are builders - they have noun names and return values


The Builder Pattern and its concerns


In [4]:
// Builder Pattern example (though the author recommends against it)
class Book (
  val author: String = "",
  val title: String = "",
  val pages: List[String] = List.empty
) {
  def withAuthor(author: String): Book = {
    new Book(author, title, pages)
  }

  def withTitle(title: String): Book = {
    new Book(author, title, pages)
  }

  def withPage(page: String): Book = {
    new Book(author, title, pages :+ page)
  }
}

// Usage
val myBook = new Book()
  .withAuthor("John Doe")
  .withTitle("Clean Code")
  .withPage("Chapter 1")

defined class Book
myBook: Book = ammonite.$sess.cmd4$Helper$Book@18ef9967

The Builder Pattern methods follow the principle - they have noun-like names (with a prefix) and return values. However, as noted in the text, the author generally advises against this pattern as it often leads to less cohesive, larger objects.


A better approach to complex objects with many properties


In [ ]:
// Instead of a large Book class with a builder, break it into smaller objects
class Author(val name: String)
class Title(val text: String)
class Page(val content: String)

class Book(val author: Author, val title: Title, val pages: List[Page]) {
  def printInfo(): Unit = {
    println(s"${title.text} by ${author.name}, ${pages.size} pages")
  }
}

// Usage
val book = new Book(
  new Author("Jane Smith"),
  new Title("Elegant Objects"),
  List(new Page("Introduction"), new Page("Chapter 1"))
)
book.printInfo()

This approach creates smaller, more focused objects rather than using the Builder Pattern for complex initialization.


## Why This Principle Matters

This strict separation between builders and manipulators makes code:

1. **More readable** - Method names clearly indicate what they do
2. **More predictable** - Builders don't change state, manipulators don't return values
3. **More maintainable** - Single responsibility principle applied at the method level
4. **More testable** - Easier to test methods with clear, focused responsibilities


## Handling Complex Cases

Sometimes we need both information and manipulation. The solution is to create specific objects for each responsibility:


In [5]:
// Example: Managing a database connection
class Database(url: String) {
  // Builder - returns a connection object
  def connection: Connection = {
    new Connection(url)
  }

  // Builder - returns statistics object
  def stats: Statistics = {
    new Statistics(url)
  }
}

class Connection(url: String) {
  // Manipulator - executes a query
  def execute(sql: String): Unit = {
    println(s"Executing $sql on $url")
  }

  // Manipulator - closes the connection
  def close(): Unit = {
    println("Connection closed")
  }
}

class Statistics(url: String) {
  // Builder - returns query count
  def queryCount: Int = {
    // Implementation to get query count
    42
  }

  // Builder - returns connection time
  def connectionTime: Long = {
    // Implementation to get connection time
    1000L
  }
}

// Usage
val db = new Database("jdbc:postgresql://localhost:5432/mydb")
val conn = db.connection
conn.execute("SELECT * FROM users")
conn.close()

val stats = db.stats
println(s"Executed ${stats.queryCount} queries")
println(s"Connection time: ${stats.connectionTime}ms")


Executing SELECT * FROM users on jdbc:postgresql://localhost:5432/mydb
Connection closed
Executed 42 queries
Connection time: 1000ms


defined class Database
defined class Connection
defined class Statistics
db: Database = ammonite.$sess.cmd5$Helper$Database@7e2ce878
conn: Connection = ammonite.$sess.cmd5$Helper$Connection@194543a4
stats: Statistics = ammonite.$sess.cmd5$Helper$Statistics@6d254bb0

### Boolean Results

Boolean methods are an exception to the naming rules. They are builders (they return values), but instead of using nouns, they should use adjectives:


In [6]:
class File(path: String) {
  // Good: Using adjectives for Boolean methods
  def empty: Boolean = {
    // Implementation to check if file is empty
    true
  }

  def readable: Boolean = {
    // Implementation to check if file is readable
    true
  }

  def present: Boolean = {
    // Better than "exists" - "is present" sounds right
    true
  }

  // Bad examples (would be better as adjectives)
  def isEmpty: Boolean = empty // Not recommended - redundant "is" prefix
  def exists: Boolean = present // Not recommended - use "present" instead
}

case class User(name: String, age: Int) {
  // Good: "equal to" sounds right
  def equalTo(other: User): Boolean = {
    this.name == other.name && this.age == other.age
  }

  // Good: adjective for Boolean result
  def adult: Boolean = {
    age >= 18
  }
}

// Usage examples showing natural reading in conditionals
val file = new File("/tmp/data.txt")
val user = new User("John", 25)
val otherUser = new User("Jane", 30)

if (file.empty) {
  println("The file is empty") // Reads naturally: "if file is empty"
}

if (file.readable) {
  println("The file can be read") // Reads naturally: "if file is readable"
}

if (user.adult) {
  println("User is an adult") // Reads naturally: "if user is adult"
}

if (user.equalTo(otherUser)) {
  println("Users are the same") // Reads naturally: "if user is equal to other user"
}


The file is empty
The file can be read
User is an adult


defined class File
defined class User
file: File = ammonite.$sess.cmd6$Helper$File@7ef5dca2
user: User = User(name = "John", age = 25)
otherUser: User = User(name = "Jane", age = 30)

## Summary of Boolean Method Naming Rules

1. **Boolean methods are builders** (they return values)
2. **Use adjectives for Boolean methods**, not nouns or verbs
3. **Don't use the "is" prefix** in the method name
4. **Test readability** by mentally placing "is" before the name - it should sound correct
5. **Examples of good names**:
   - `empty()` instead of `isEmpty()`
   - `readable()` instead of `canRead()`
   - `negative()` instead of `isNegative()`
   - `equalTo(obj)` instead of `equals(obj)`
   - `present()` instead of `exists()`

This naming convention makes conditionals read more naturally, as the Boolean methods essentially answer the question "is this object [adjective]?"

## 2.5 Don't use public constants

Public constants create global coupling and reduce cohesion.
Prefer keeping values private to the class that uses them, or pass them in as small objects (dependencies) instead of sharing a global constant.

In [11]:
// Bad example: global constant-like object (avoid)
object ConstantsBad {
  val CrLf: String = "\r\n"
}

class RowsBad2(private val all: List[String]) {
  def print(p: java.io.PrintStream): Unit = {
    all.foreach { row => p.print(s"$row${ConstantsBad.CrLf}") }
  }
}

defined object ConstantsBad
defined class RowsBad2

Better alternatives:

- Keep the value private and local to the class.
- Or encapsulate the value as an object and inject it.

In [12]:
class Rows2(private val all: List[String]) {
  private val CrLf: String = "\r\n"
  def print(p: java.io.PrintStream): Unit =
    all.foreach { row => p.print(s"$row$CrLf") }
}

trait LineSeparator { def value: String }
class CrLf extends LineSeparator { val value = "\r\n" }
class Lf extends LineSeparator { val value = "\n" }

class Records2(private val all: List[String], private val sep: LineSeparator) {
  def write(out: java.io.Writer): Unit = {
    all.foreach { rec =>
      out.write(rec)
      out.write(sep.value)
    }
  }
}

defined class Rows2
defined trait LineSeparator
defined class CrLf
defined class Lf
defined class Records2

## 2.5.1 Introduction of coupling

Public constants introduce hidden, hard-coded dependencies. When multiple classes reach out to a shared global value, they become tightly coupled to it and to each other, making behavior changes unpredictable.

Consider these two classes that both rely on a shared line separator constant:


In [13]:
// Bad: shared global constant couples unrelated classes to the same value
object ConstantsGlobal {
  val CrLf: String = "\r\n"
}

class RecordsCoupled(private val all: List[String]) {
  def write(out: java.io.Writer): Unit = {
    all.foreach { rec =>
      out.write(rec)
      out.write(ConstantsGlobal.CrLf) // hard dependency here
    }
  }
}

class RowsCoupled(private val all: List[String]) {
  def print(p: java.io.PrintStream): Unit = {
    all.foreach { row =>
      p.print(s"$row${ConstantsGlobal.CrLf}") // hard dependency here
    }
  }
}

defined object ConstantsGlobal
defined class RecordsCoupled
defined class RowsCoupled

Now, RecordsCoupled.write, RowsCoupled.print, and ConstantsGlobal.CrLf are all coupled. If we change CrLf (for example, to a platform-specific or protocol-specific value), the behavior of both classes changes, and it’s unclear whether those changes are acceptable. One user may be printing to the console, while another may be composing an HTTP message where the line ending is mandated and must not change.

The constant sits in a global scope without semantic context. We can’t tell how it���s used or which usages are safe to alter. This erodes maintainability: many objects depend on a value whose meaning varies by context.

A better design is to inject small objects that carry the context (as shown above with LineSeparator and Records2/Rows2), so each class depends on what it needs, and changes are localized.


## 2.5.2 Loss of cohesion

Public constants push unrelated semantics into consumers. Records and Rows shouldn’t know how to apply a line ending; that’s a separate responsibility. Instead of sharing a naked value, share behavior with a tiny object that owns the meaning of the value.

Key ideas:
- Don’t share data, share behavior (micro classes).
- Couple through contracts, not through globals.
- Semantics live with the object that knows them.


In [23]:
// A tiny object that owns the semantics of “append CRLF to a line”
final case class CRLFString(private val origin: String) {
  override def toString: String = s"$origin\r\n"
}

// Usage in code that writes records (no constant, no formatting logic here)
final class Records3(private val all: List[String]) {
  def write(out: java.io.Writer): Unit =
    all.foreach { rec =>
      out.write(CRLFString(rec))
    }
}


defined class CRLFString
defined class Records3

In [25]:
// Usage in code that prints rows (PrintStream can print any object via toString)
final class Rows3(private val all: List[String]) {
  def print(p: java.io.PrintStream): Unit =
    all.foreach { row =>
      p.print(CRLFString(row))
    }
}

private val rows = new Rows3(List("row1", "row2"))
rows.print(System.out) // prints with CRLF


row1
row2


defined class Rows3

Platform-aware behavior stays encapsulated in the same micro class. The contract doesn’t change; only behavior does.


In [29]:
final class SafeCRLFString(private val origin: String) {
  override def toString: String = {
    val os = java.lang.System.getProperty("os.name", "").toLowerCase
    if (os.contains("win"))
      throw new IllegalStateException("We're on Windows, can't use CRLF, sorry")
    s"$origin\r\n"
  }
}

object SafeCRLFString {
  def apply(origin: String): SafeCRLFString = new SafeCRLFString(origin)
}


defined class SafeCRLFString
defined object SafeCRLFString

Another example: replace HTTP method constants with tiny classes that configure a request.


In [ ]:
// A very small HTTP request stub; the method is part of its state
trait Request { def fetch(): String }

final class HttpRequest extends Request {
  private var method: String = "GET"
  def withMethod(m: String): HttpRequest = { method = m; this }
  override def fetch(): String = s"Fetched with method=$method"
}

// Instead of HttpMethods.POST constant, use a tiny configurator object
final class PostRequest(private val origin: HttpRequest) extends Request {
  override def fetch(): String = origin.withMethod("POST").fetch()
}

// Usage
val bodyViaPost: String = new PostRequest(new HttpRequest).fetch()

## 2.6 Be immutable

Make classes immutable to keep them small, cohesive, decoupled, and easy to reason about. An immutable object doesn’t change its state after creation; operations return new objects instead of mutating existing ones.

Key ideas:
- Prefer vals over vars; keep fields private and final by default.
- Methods that “modify” return new instances (no setters).
- Small, focused objects compose behavior instead of sharing mutable state.


### Mutable vs. Immutable example (money arithmetic)


In [ ]:
// Bad: mutable object with in-place modification
final class MutableMoney(private var dollars: Int) {
  def mul(factor: Int): Unit = { dollars *= factor }
  override def toString: String = s"$$$dollars"
}

val fiveMut = new MutableMoney(5)
fiveMut.mul(10)
println(fiveMut) // "$50" — original object changed


In [32]:
// Good: immutable object that returns a new instance
final case class ImmutableMoney(private val dollars: Int) {
  def mul(factor: Int): ImmutableMoney = copy(dollars = dollars * factor)
  override def toString: String = s"$$$dollars"
}

val five = ImmutableMoney(5)
val fifty = five.mul(10)
println(five)   // "$5"
println(fifty)  // "$50"


$5
$50


defined class ImmutableMoney
five: ImmutableMoney = ImmutableMoney(dollars = 5)
fifty: ImmutableMoney = ImmutableMoney(dollars = 50)

Why this is better:
- No hidden state changes; values read like they are named.
- Easier to test and reason about; no ordering surprises.
- Safer to share across threads (no synchronization needed for reads).


### Practical rules of thumb for immutability in Scala

- Use case classes and vals; avoid public vars and setters.
- Prefer tiny value objects over exposing raw primitives.
- Keep constructors doing all necessary initialization; don’t leave objects half-baked.
- If you need a different value, construct a different object.


### Lazy loading without mutability

Lazy loading is often cited as a reason to mutate. In Scala, lazy val gives you safe, one-time initialization without exposing mutability.


In [35]:
// A source for HTML content
trait HtmlSource { def fetch(uri: String): String }

// Immutable page with lazy-loaded content
final class PageLazy(private val uri: String, private val src: HtmlSource) {
  // Thread-safe, computed once on first access, then cached
  lazy private val html: String = src.fetch(uri)
  def content: String = html
}

final class StubSource extends HtmlSource {
  override def fetch(uri: String): String = s"<html><body>Loaded $uri</body></html>"
}

val page = new PageLazy("https://example.org", new StubSource)
println(page.content) // triggers fetch once
println(page.content) // returns cached result


<html><body>Loaded https://example.org</body></html>
<html><body>Loaded https://example.org</body></html>


defined trait HtmlSource
defined class PageLazy
defined class StubSource
page: PageLazy = ammonite.$sess.cmd35$Helper$PageLazy@30ab8e82

Notes:
- lazy val in Scala is thread-safe and evaluates at most once per instance.
- Keep dependencies small (like HtmlSource) and inject them; PageLazy stays cohesive and testable.


### A tiny, reusable memoization helper

For computed values that aren’t fields, wrap the computation in a tiny object and memoize it.


In [36]:
final class Memoized[A](thunk: => A) {
  // Evaluate on first access and cache thereafter
  lazy val value: A = thunk
}

// Usage: any expensive computation
def heavy(n: Int): Int = { println(s"computing for $n"); n * n }

val squared = new Memoized(heavy(21))
println(squared.value) // computes
println(squared.value) // cached


computing for 21
441
441


defined class Memoized
defined function heavy
squared: Memoized[Int] = ammonite.$sess.cmd36$Helper$Memoized@23e43f74

### Takeaways

- Don’t mutate objects; return new ones.
- Use lazy val for one-time, cached initialization without sacrificing immutability.
- Prefer micro-objects and explicit dependencies over shared mutable state.

### 2.6.1 Identity mutability

Immutable objects don’t suffer from “identity mutability.” The bug appears when an object’s equality and hash code depend on mutable state: after the object is used as a key in a map (or element of a set), mutating it breaks the collection’s invariants. Lookups become unpredictable and duplicates (by equality) may appear.


In [37]:
import scala.collection.mutable

// Bad: key equality depends on mutable state
final class MutableKeyMoney(private var _dollars: Int) {
  def dollars: Int = _dollars
  def mul(factor: Int): Unit = { _dollars *= factor }

  // Equality and hashCode depend on (mutable) dollars
  override def equals(obj: Any): Boolean = obj match {
    case that: MutableKeyMoney => this._dollars == that._dollars
    case _ => false
  }
  override def hashCode(): Int = java.lang.Integer.hashCode(_dollars)
  override def toString: String = s"$$$dollars"
}

val map = mutable.HashMap[MutableKeyMoney, String]()
val fiveK = new MutableKeyMoney(5)
val tenK  = new MutableKeyMoney(10)
map.put(fiveK, "five")
map.put(tenK,  "ten")

// Mutate one key after it has been placed in the map
fiveK.mul(2) // now equals(tenK) and hashCode matches tenK

println(map)                  // keys may look equal ($10, $10), but both entries still exist internally
println(map.get(fiveK))       // lookup may fail or be inconsistent (hash bucket mismatch)
println(map.get(tenK))        // may succeed, highlighting the broken invariant
println(map.keys.count(k => k == fiveK)) // > 1 by equality, revealing duplicates by equals()


HashMap($10 -> five, $10 -> ten)
Some(ten)
Some(ten)
2


import scala.collection.mutable

// Bad: key equality depends on mutable state

defined class MutableKeyMoney
map: HashMap[MutableKeyMoney, String] = HashMap($10 -> "five", $10 -> "ten")
fiveK: MutableKeyMoney = $10
tenK: MutableKeyMoney = $10
res37_5: Option[String] = None
res37_6: Option[String] = None

Why this happens:
- Hash-based collections place keys into buckets using hashCode computed at insertion time.
- If hashCode/equals depend on mutable fields and those fields change after insertion, the stored bucket no longer matches the key’s identity.
- Future lookups use the new hash/equality and search the wrong bucket; duplicates-by-equality can also appear.


Safe alternative: immutable keys (stable equality)


In [5]:
import scala.collection.mutable

// Good: immutable key with stable equals/hashCode
final case class MoneyKey(dollars: Int) {
  override def toString: String = s"$$$dollars"
}

val safe = mutable.HashMap[MoneyKey, String]()
safe.put(MoneyKey(5),  "five")
safe.put(MoneyKey(10), "ten")

// We “change” by creating a new key; the old entry remains valid
val fiveKey = MoneyKey(5)
safe.put(fiveKey.copy(dollars = fiveKey.dollars * 2), "ten-again")

println(safe)                 // Map($5 -> five, $10 -> ten-again)
println(safe.get(MoneyKey(10))) // deterministic and consistent


HashMap($5 -> five, $10 -> ten-again)
Some(ten-again)


import scala.collection.mutable

// Good: immutable key with stable equals/hashCode

defined class MoneyKey
safe: HashMap[MoneyKey, String] = HashMap(
  MoneyKey(dollars = 5) -> "five",
  MoneyKey(dollars = 10) -> "ten-again"
)
res5_3: Option[String] = None
res5_4: Option[String] = None
fiveKey: MoneyKey = MoneyKey(dollars = 5)
res5_6: Option[String] = Some(value = "ten")

Guidelines to avoid identity mutability bugs:
- Never use mutable objects as keys in HashMap/HashSet (or any hashed structure).
- Ensure equals/hashCode rely only on immutable state. Prefer case classes for value objects.
- Keep classes final with private vals; expose behavior, not setters.
- “Modifications” should produce new objects, leaving existing map keys valid forever.


### 2.6.2 Failure atomicity

Failure atomicity means an operation either succeeds completely or it fails leaving the object exactly as it was before the attempt. Immutable objects give you this for free: since they never mutate internal state, either a new object is created (success) or an exception is thrown (failure) and the original object remains intact. Mutable objects, on the other hand, can easily end up half‑updated when an exception is raised mid‑way through a method.

We will contrast three implementations:
1. A naive mutable implementation that can leave the object inconsistent on failure.
2. An immutable version that is naturally failure atomic.
3. A mutable implementation that manually restores prior state (higher complexity / risk).

#### 1. Naive mutable object (partial update on failure)

In [3]:
final class CashMutablePartial(private var dollars: Int, private var cents: Int) {
  // Multiplies both parts; negative factor simulates a validation failure mid-way
  def mul(factor: Int): Unit = {
    dollars *= factor              // (1) state already changed
    if (factor < 0) {              // simulate a validation problem discovered after first change
      throw new IllegalArgumentException("factor must be non-negative")
    }
    cents *= factor                // (2) never reached if exception above; cents stays old value
  }
  override def toString: String = f"$$$dollars%d.$cents%02d"
}

val partialCash = new CashMutablePartial(10, 50)
println(s"Before (partial): $partialCash")
try partialCash.mul(-2) catch { case e: Exception => println(s"Failed: ${e.getMessage}") }
println(s"After failure (partial):  $partialCash  // dollars changed, cents not -> inconsistent state")

Before (partial): $10.50
Failed: factor must be non-negative
After failure (partial):  $-20.50  // dollars changed, cents not -> inconsistent state


defined class CashMutablePartial
partialCash: CashMutablePartial = $-20.50

The object is left in a broken, intermediate state: dollars changed; cents did not. Any subsequent logic now operates on corrupt data.

#### 2. Immutable version (automatic failure atomicity)

In [2]:
final case class CashImmutable(dollars: Int, cents: Int) {
  def mul(factor: Int): CashImmutable = {
    if (factor < 0) throw new IllegalArgumentException("factor must be non-negative")
    copy(dollars = dollars * factor, cents = cents * factor)
  }
  override def toString: String = f"$$$dollars%d.$cents%02d"
}

val original = CashImmutable(10, 50)
println(s"Original (immutable): $original")
val twenty = original.mul(2)
println(s"After success (immutable): $twenty (original still $original)")
try original.mul(-3) catch { case e: Exception => println(s"Immutable failed, original preserved: $original (${e.getMessage})") }

Original (immutable): $10.50
After success (immutable): $20.100 (original still $10.50)
Immutable failed, original preserved: $10.50 (factor must be non-negative)


defined class CashImmutable
original: CashImmutable = CashImmutable(dollars = 10, cents = 50)
twenty: CashImmutable = CashImmutable(dollars = 20, cents = 100)
res2_5: Helper.this.CashImmutable | scala.Unit = ()

Because the immutable method creates and returns a new object, either it succeeds (returning a new consistent instance) or it throws, leaving the caller's reference untouched.

#### 3. Manually failure-atomic mutable implementation

In [1]:
final class CashMutableAtomic(private var dollars: Int, private var cents: Int) {
  def mul(factor: Int): Unit = {
    // Snapshot old state
    val beforeD = dollars
    val beforeC = cents
    try {
      dollars = dollars * factor
      if (factor < 0) throw new IllegalArgumentException("factor must be non-negative")
      cents = cents * factor
    } catch {
      case e: Exception =>
        // Roll back all mutated fields
        dollars = beforeD
        cents = beforeC
        throw e
    }
  }
  override def toString: String = f"$$$dollars%d.$cents%02d"
}

val atomic = new CashMutableAtomic(10, 50)
println(s"Before (atomic): $atomic")
try atomic.mul(-4) catch { case e: Exception => println(s"Rolled back after failure: $atomic (${e.getMessage})") }
atomic.mul(3)
println(s"After success (atomic):  $atomic")

Before (atomic): $10.50
Rolled back after failure: $10.50 (factor must be non-negative)
After success (atomic):  $30.150


defined class CashMutableAtomic
atomic: CashMutableAtomic = $30.150

This version preserves invariants by copying and restoring prior state on failure. It works, but at a cost:

- Boilerplate grows with the number of fields.
- Easy to miss a field during rollback (especially after future modifications).
- Harder to read; exception handling intertwines with business logic.
- Encourages adding more state (since mutation seems "managed"), increasing risk.

Immutable design avoids all of this incidental complexity.

#### Takeaways (Failure Atomicity)

1. Immutable objects are inherently failure atomic: no partial mutations can leak out.
2. Naive mutable code often leaves objects half-updated when exceptions occur.
3. Achieving failure atomicity in mutable objects requires defensive snapshots and rollback logic (error-prone / verbose).
4. Prefer immutable value objects (case classes) for operations that transform state; return new instances.
5. If mutability is unavoidable (performance / interoperability), isolate it behind a tiny API and rigorously maintain invariants.

Failure atomicity reduces debugging time, increases trust in invariants, and keeps methods focused on domain logic instead of state management scaffolding.

### 2.6.3 Temporal coupling

Temporal coupling happens when the correctness of code depends on the chronological order of separate statements that manipulate the same (mutable) object. The object is born incomplete, then gradually “filled in” via setters or mutators. Rearranging or omitting any of those statements silently breaks logic while still compiling.

Immutability eliminates this category of risk by forcing full initialization at construction time: one expression creates a complete, valid object. There is nothing to “remember” about sequencing afterwards.


#### 1. Mutable (JavaBean‑style) object requiring ordered setters


In [10]:
// A mutable bean-like Cash with temporal coupling in its usage
final class CashBean {
  private var dollars: Int = 0     // starts uninitialized (semantic NULL)
  private var cents: Int   = 0
  def setDollars(d: Int): Unit = { dollars = d }
  def setCents(c: Int): Unit = { cents = c }
  override def toString: String = f"$$${dollars}%d.${cents}%02d"
}

// Correct order (instantiate, then set both, then use)
val bean = new CashBean()
bean.setDollars(29)
bean.setCents(95)
println(s"Correct:  $bean")  // "$29.95"


Correct:  $29.95


defined class CashBean
bean: CashBean = $29.95

Reordering still compiles, but produces the wrong result because printing happens before full initialization:


In [11]:
val beanWrong = new CashBean()
beanWrong.setDollars(29)
println(s"Printed too early:  $beanWrong") // "$29.00" (cents not set yet)
beanWrong.setCents(95)                     // Late; no compiler/runtime warning


Printed too early:  $29.00


beanWrong: CashBean = $29.95

#### 2. Temporal coupling amplified by distance

Distance (other logic between setter calls) makes coupling harder to spot:


In [15]:
val distant = new CashBean()
// 1) some unrelated logic (pretend this block is large & complicated)
val xs = (1 to 100).map(_ * 2).sum // noise before real initialization
// 2) first part of initialization
val x = 42 // pretend we computed this; irrelevant
// Set dollars derived from some earlier computation
val computedDollars = 17 + 12
distant.setDollars(computedDollars)

// 50 lines later (imagine transformations, conditionals, loops, logging, etc.)
val ys = (1 to 50).filter(_ % 7 == 0).sum // more noise
// Only now we finally remember to set cents
val computedCents = 95
distant.setCents(computedCents)

// If someone moves this print higher (above setCents) they introduce a bug that still compiles
println(s"Distant: $distant")


Distant: $29.95


distant: CashBean = $29.95
xs: Int = 10100
x: Int = 42
computedDollars: Int = 29
ys: Int = 196
computedCents: Int = 95

At every maintenance step, a reader must mentally reconstruct the implicit ordering contract: “all setters must run before any use.” The compiler can’t help.

Problems:
- Objects exist in a transient, invalid state for a period of time.
- Readability & refactor safety degrade as distance grows.
- Partial initialization bugs are silent (no type signal, no runtime exception).
- Parallelizing or reordering logic becomes risky.


#### 3. Immutable value object (no temporal coupling)


In [18]:
// Immutable: fully initialized at construction, validated once
final case class CashImmutableTemporal(dollars: Int, cents: Int) {
  require(dollars >= 0, "dollars must be non-negative")
  require(cents >= 0 && cents < 100, "cents must be 0..99")
  override def toString: String = f"$$$dollars%d.$cents%02d"

  // Transformation returns a NEW object instead of mutating
  def plus(other: CashImmutableTemporal): CashImmutableTemporal = {
    val totalCents = this.cents + other.cents
    val carry = totalCents / 100
    CashImmutableTemporal(this.dollars + other.dollars + carry, totalCents % 100)
  }
}

val price  = CashImmutableTemporal(29, 95)
val tax    = CashImmutableTemporal( 0,  80)
val total  = price.plus(tax)
println(s"Price: $price  Tax: $tax  Total: $total")


Price: $29.95  Tax: $0.80  Total: $30.75


defined class CashImmutableTemporal
price: CashImmutableTemporal = CashImmutableTemporal(dollars = 29, cents = 95)
tax: CashImmutableTemporal = CashImmutableTemporal(dollars = 0, cents = 80)
total: CashImmutableTemporal = CashImmutableTemporal(dollars = 30, cents = 75)

You can’t “forget” to set cents: the constructor forces both dollars and cents to be provided. There is no legal intermediate state, and usage order is irrelevant beyond standard data dependencies.

Benefits:
- Construction is atomic: one expression yields a valid object.
- Order independence: no hidden sequencing contract to remember.
- Fewer states to test (no half-initialized permutations).
- Referential transparency of transformations (price.plus(tax) is pure).


#### 4. Eliminating temporal coupling with micro value objects

Even if values come from distant computations, produce tiny immutable intermediates and compose them once:


In [22]:
final case class Dollars(value: Int) { require(value >= 0) }
final case class Cents(value: Int)   { require(value >= 0 && value < 100) }
final case class Price(parts: (Dollars, Cents)) {
  private val (d, c) = parts
  override def toString: String = f"$$${d.value}%d.${c.value}%02d"
}

// Simulate distant computations
val dPart: Dollars = { /* many lines... */ Dollars(29) }
val cPart: Cents   = { /* many more lines... */ Cents(95) }
// Composition happens once, at the point of actual need
val composed = Price(dPart -> cPart)
println(s"Composed price: $composed")


Composed price: $29.95


defined class Dollars
defined class Cents
defined class Price
dPart: Dollars = Dollars(value = 29)
cPart: Cents = Cents(value = 95)
composed: Price = Price(parts = (Dollars(value = 29), Cents(value = 95)))

Each sub-object is itself already valid; final assembly remains a single, obvious step. There is still no temporal fragility—reordering dPart & cPart computations (if independent) does not risk a half-baked Price.


#### 5. Takeaways (Temporal Coupling)

1. Mutable “bean” patterns force readers to memorize a correct sequence (instantiate → set all fields → use). That sequence is invisible to the type system.
2. Distance between setter calls hides bugs and discourages refactoring and reordering.
3. Immutable constructors collapse instantiation + initialization into a single, enforced step.
4. Pure transformation methods (returning new objects) keep objects always valid—no transitional states.
5. Micro value objects (Dollars, Cents) let you spread computations without re-introducing temporal coupling.
6. Less temporal coupling → easier parallelization, testing, and reasoning; fewer timing / order-of-operations defects.

In short: immutability erases an entire class of ordering problems; the compiler helps you maintain invariants instead of relying on human memory.

### 2.6.4 Side effect-free

A side effect is any observable mutation that escapes the scope where it happens. Mutable objects make accidental side effects easy: a method that *looks* like a pure printer can silently change its argument. This increases debugging time because every caller becomes a suspect.

In [23]:
// Mutable example: printing also mutates the argument (hidden side effect)
final class CashSide(private var dollars: Int) {
  def mul(f: Int): Unit = { dollars *= f }
  override def toString: String = s"$$$dollars"
}

def printBad(price: CashSide): Unit = {
  println(s"Today price is: $price")
  price.mul(2)                // sneaky mutation
  println(s"Buy now, tomorrow price is: $price")
}

val fiveSide = new CashSide(5)
printBad(fiveSide)
println(s"After call (oops, changed): $fiveSide")

Today price is: $5
Buy now, tomorrow price is: $10
After call (oops, changed): $10


defined class CashSide
defined function printBad
fiveSide: CashSide = $10

Because `printBad` mutates its parameter, the caller observes an unexpected new value. Tracing this in a large code base is costly.

In [25]:
// Immutable alternative: no hidden mutation, transformation is explicit
final case class CashPure(dollars: Int) {
  def mul(f: Int): CashPure = copy(dollars = dollars * f) // returns a NEW value
  override def toString: String = s"$$$dollars"
}

def printPure(price: CashPure): Unit = {
  println(s"Today price is: $price")
  val tomorrow = price.mul(2)      // explicit new value
  println(s"Buy now, tomorrow price is: $tomorrow")
}

val fivePure = CashPure(5)
printPure(fivePure)
println(s"After call (still original): $fivePure")

Today price is: $5
Buy now, tomorrow price is: $10
After call (still original): $5


defined class CashPure
defined function printPure
fivePure: CashPure = CashPure(dollars = 5)

Takeaways (Side effect-free):
- Hidden mutations create debugging noise; every use-site must be audited.
- Immutability guarantees arguments remain trustworthy after calls.
- Methods either return a new value (builder) or just act (manipulator) — never both.
- Favor explicit returned results over in-place modification; it localizes change and prevents surprises.

## 2.6.5 No NULL references
NULL (or null) hides missing design: a field that can be null usually means an object is being used in more than one role. Instead of temporary "unset" state, model the absence explicitly or split responsibilities.

In [26]:
// Bad: nullable, mutable field with ad-hoc null checks
final class MutableUserBad(private val id: Int) {
  private var name: String = null // sentinel state
  def setName(n: String): Unit = { name = n }
  def greet(): String = if (name != null) s"Hello, $name" else "Hello" // repetitive null logic
}

val bad = new MutableUserBad(1)
println(bad.greet())      // "Hello" (anonymous by implicit null)
bad.setName("Alice")
println(bad.greet())      // "Hello, Alice" (state meaning changed)

Hello
Hello, Alice


defined class MutableUserBad
bad: MutableUserBad = ammonite.$sess.cmd26$Helper$MutableUserBad@f239e4d

Better: force construction with all required data (immutability ⇒ no null possible).

In [27]:
final case class PersonName(value: String) // tiny value object
final case class Person(id: Int, name: PersonName) {
  def greet: String = s"Hello, ${name.value}" // no conditional logic
}

val p = Person(1, PersonName("Alice"))
println(p.greet)

Hello, Alice


defined class PersonName
defined class Person
p: Person = Person(id = 1, name = PersonName(value = "Alice"))

If absence is a real, domain concept, model it explicitly instead of encoding it as null.

In [29]:
sealed trait PersonRef { def greet: String }
final case class NamedPerson(id: Int, name: PersonName) extends PersonRef {
  override def greet: String = s"Hello, ${name.value}" }
final case class AnonymousPerson(id: Int) extends PersonRef {
  override def greet: String = "Hello" }

val people: List[PersonRef] = List(NamedPerson(2, PersonName("Bob")), AnonymousPerson(3))
people.foreach(p => println(p.greet))

Hello, Bob
Hello


defined trait PersonRef
defined class NamedPerson
defined class AnonymousPerson
people: List[PersonRef] = List(
  NamedPerson(id = 2, name = PersonName(value = "Bob")),
  AnonymousPerson(id = 3)
)

Option is acceptable when a value is truly optional, but keep it at the boundary—prefer distinct types when behavior changes.

In [30]:
final case class PersonWithOption(id: Int, name: Option[PersonName]) {
  def greet: String = name.fold("Hello")(n => s"Hello, ${n.value}")
}
println(PersonWithOption(4, None).greet)
println(PersonWithOption(5, Some(PersonName("Eve"))).greet)

Hello
Hello, Eve


defined class PersonWithOption

Takeaways (No NULL references):
- Null encodes “missing type design,” not just “missing data.”
- Immutability + mandatory constructor params eliminate accidental nulls.
- Use distinct tiny classes (Named/Anonymous) or Option for true optionality.
- Fewer sentinel states ⇒ simpler, safer, more maintainable code.

## 2.6.6 Thread safety
Mutable shared state + unsynchronized writes = race conditions and transiently broken invariants.

In [ ]:
// Not thread-safe: two fields updated in two steps
final class MutableCashTS(private var dollars: Int, private var cents: Int) {
  def mul(f: Int): Unit = { // race window between the two writes
    dollars *= f
    // Simulate a bit of work to enlarge the race window
    Thread.`yield`()
    cents *= f
  }
  override def toString: String = f"$$$dollars%d.$cents%02d"
}

val shared = new MutableCashTS(15, 10)
val latch  = new java.util.concurrent.CountDownLatch(1)
val pool   = java.util.concurrent.Executors.newFixedThreadPool(4)
(1 to 2).foreach { _ =>
  pool.submit(new Runnable { override def run(): Unit = { latch.await(); shared.mul(2); println(shared) } })
}
latch.countDown() // start both threads
pool.shutdown()

You can occasionally observe an impossible state (like $60.20) where only one field was updated twice.

### Synchronized fix (works, but adds contention)

In [ ]:
final class MutableCashSync(private var dollars: Int, private var cents: Int) {
  def mul(f: Int): Unit = this.synchronized {
    dollars *= f
    Thread.sleep(2) // exaggerate lock holding
    cents *= f
  }
  override def toString: String = f"$$$dollars%d.$cents%02d"
}

val syncShared = new MutableCashSync(15, 10)
val latch2  = new java.util.concurrent.CountDownLatch(1)
val pool2   = java.util.concurrent.Executors.newFixedThreadPool(2)
(1 to 2).foreach { _ => pool2.submit(new Runnable { override def run(): Unit = { latch2.await(); syncShared.mul(2); println(s"sync: $syncShared") } }) }
latch2.countDown()
pool2.shutdown()

Locks serialize access; safe but slower and prone to deadlocks in more complex graphs.

### Immutable alternative (inherently thread-safe)

In [ ]:
import scala.concurrent.{Future, Await}
import scala.concurrent.duration.*
import scala.concurrent.ExecutionContext.Implicits.global

final case class CashImm(dollars: Int, cents: Int) {
  def mul(f: Int): CashImm = copy(dollars = dollars * f, cents = cents * f)
  override def toString: String = f"$$$dollars%d.$cents%02d"
}

val base = CashImm(15, 10)
val factors = List(2,2)
val futures = Future.traverse(factors)(f => Future(base.mul(f)))
val results = Await.result(futures, 2.seconds)
println(s"Original: $base  Results: ${results.mkString(", ")}")

Each thread works on its own new object; no shared mutation, no races, no locks.

Takeaways (Thread safety):
- Races stem from interleaved writes to shared mutable state.
- Synchronization fixes correctness but adds contention & complexity.
- Immutability sidesteps the problem: no in-place mutation to protect.
- Prefer designing objects so concurrent use only reads shared data or creates new values.
